In [ ]:
from janus import JANUS
from rdkit import RDLogger

from Source.models.GCNN_FCNN.featurizers import Complex
from Source.models.GCNN_FCNN.model import GCNN_FCNN
from Source.trainer import ModelShell
from config import ROOT_DIR

RDLogger.DisableLog("rdApp.*")
import selfies

METAL_1 = "Am"
METAL_2 = "Cm"
VALENCE = 3
TEMPERATURE = 25
IONIC_STR = 0.1
MODEL = ModelShell(GCNN_FCNN, ROOT_DIR / "App_models/Y_Sc_f-elements_5fold_regression_2023_06_10_07_58_17")
WORKING_DIR = "../Generation_results"
STARTING_POPULATION_PATH = "../Data/nist_start_smiles.txt"

print(selfies.get_semantic_constraints())

def fitness_function(smi: str) -> float:
    """ User-defined function that takes in individual smiles
    and outputs a fitness value.
    """
    complex_1 = Complex(mol=smi, metal=METAL_1, valence=VALENCE, temperature=TEMPERATURE, ionic_str=IONIC_STR)
    complex_2 = Complex(mol=smi, metal=METAL_2, valence=VALENCE, temperature=TEMPERATURE, ionic_str=IONIC_STR)

    return MODEL(complex_1.graph)["logK"].item() / MODEL(complex_2.graph)["logK"].item()


def custom_filter(smi: str):
    """ Function that takes in a smile and returns a boolean.
    True indicates the smiles PASSES the filter.
    """
    # smiles length filter
    if len(smi) > 120 or len(smi) < 20:
        return False
    elif "+" in smi or "-" in smi:
        return False
    else:
        return True


# all parameters to be set, below are defaults
params_dict = {
    # Number of iterations that JANUS runs for
    "generations": 20,

    # The number of molecules for which fitness calculations are done,
    # exploration and exploitation each have their own population
    "generation_size": 1000,

    # Number of molecules that are exchanged between the exploration and exploitation
    "num_exchanges": 5,

    # Callable filtering function (None defaults to no filtering)
    "custom_filter": custom_filter,

    # Fragments from starting population used to extend alphabet for mutations
    "use_fragments": False, # Causes errors in selfies encoding

    # An option to use a classifier as selection bias
    "use_classifier": True,
}

# Set your SELFIES constraints (below used for manuscript)
default_constraints = selfies.get_semantic_constraints()
new_constraints = default_constraints
new_constraints['S'] = 6
new_constraints['P'] = 5
selfies.set_semantic_constraints(new_constraints)  # update constraints

# Create JANUS object.
agent = JANUS(
    work_dir=WORKING_DIR,  # where the results are saved
    fitness_function=fitness_function,  # user-defined fitness for given smiles
    start_population=STARTING_POPULATION_PATH,  # file with starting smiles population
    **params_dict
)

agent.run()  # RUN IT!

In [ ]:
from rdkit import Avalon
Avalon